# COMPARE VANILLA AND TORCH IMPLEMENTATION

### PARAMETERS

Load modules and define benchmark parameters.

In [94]:
from datetime import datetime
from multiprocessing import Pool, cpu_count
import torch
from src.input.input_files import generate_input_files, file_to_matrix
import csv
from src.benchmarking.vanilla.benchmark_vanilla import check_and_benchmark as check_and_benchmark_vanilla
from src.benchmarking.torch.benchmark_torch import check_and_benchmark as check_and_benchmark_torch

mat_sizes = range(100, 115, 5)
input_gen_processors = 10

input_directory = r"resources\vanilla_torch_comparison\input_matrices"
input_file_prefix = "SingMat"

section_size = 5
benchmark_file_fieldnames = ['size', 'run', 'type', 'process_time', 'initial_rss', 'final_rss']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

output_directory = r"resources\vanilla_torch_comparison"
benchmark_file_prefix = "BenchmarkVanillaTorch"

print(f"There are {cpu_count()} threads available on this machine.")
print(f"PyTorch will be executed on device {device}.")

There are 12 threads available on this machine.
PyTorch will be executed on device cuda.


### GENERATE INPUT FILES

Generate some matrices and save them to files.

In [95]:
num_mat = len(mat_sizes)

argument_pair_list = [(mat_size, f"..\{input_directory}\{input_file_prefix}_{mat_size}_{1}.txt") for mat_size in mat_sizes]

print(f"Generating {num_mat} singular matrices ...")

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(generate_input_files, argument_pair_list)

print(f"Saved {num_mat} matrices in '{input_directory}'.")

Generating 3 singular matrices ...
Saved 3 matrices in 'resources\vanilla_torch_comparison\input_matrices'.


### GET FILES TO BENCHMARK

Find relevant files and sort them.

In [97]:
import src.input.input_files as imp

input_file_names = imp.get_txt_files(f"..\{input_directory}", f"{input_file_prefix}_")
input_file_names.sort(key = lambda file_name: imp.extract_numbers(file_name, r'_\d+_', r'_\d+\.'))

print(input_file_names)
print(f"Number of files to benchmark: {len(input_file_names)}")

['SingMat_100_1.txt', 'SingMat_105_1.txt', 'SingMat_110_1.txt']
Number of files to benchmark: 3


### BENCHMARK

Read each file into a matrix or a tensor respectively and save the results to a file.

In [98]:
benchmark_file_timestamp = datetime.now().strftime("%d%m%y_%H%M")

with open(rf'..\{output_directory}\{benchmark_file_prefix}_{benchmark_file_timestamp}.csv', mode='w', newline='') as benchmark_file:
    writer = csv.DictWriter(benchmark_file, fieldnames=benchmark_file_fieldnames)
    writer.writeheader()


for input_file_name in input_file_names:

    matrix, size, run = file_to_matrix(input_directory, input_file_name)

    print(f"Vanilla benchmark run {run} ... matrix size: {size}, section size: {section_size}")
    _, _, _, process_time_vanilla, initial_rss_vanilla, final_rss_vanilla = check_and_benchmark_vanilla(matrix, section_size)

    with open(rf'..\{output_directory}\{benchmark_file_prefix}_{benchmark_file_timestamp}.csv', mode='a', newline='') as benchmark_file:
        writer = csv.DictWriter(benchmark_file, fieldnames=benchmark_file_fieldnames)
        writer.writerow({'size': size, 'run': run, 'type': "vanilla", 'process_time': process_time_vanilla, 'initial_rss': initial_rss_vanilla, 'final_rss': final_rss_vanilla})


    tensor = torch.BoolTensor(matrix)

    print(f"Torch benchmark run {run} ... matrix size: {size}, section size: {section_size}")
    _, _, _, process_time_torch, initial_rss_torch, final_rss_torch = check_and_benchmark_torch(tensor, section_size)

    with open(rf'..\{output_directory}\{benchmark_file_prefix}_{benchmark_file_timestamp}.csv', mode='a', newline='') as benchmark_file:
        writer = csv.DictWriter(benchmark_file, fieldnames=benchmark_file_fieldnames)
        writer.writerow({'size': size, 'run': run, 'type': "torch", 'process_time': process_time_torch, 'initial_rss': initial_rss_torch, 'final_rss': final_rss_torch})

print("Done.")

Vanilla benchmark run 1 ... matrix size: 100, section size: 5
Torch benchmark run 1 ... matrix size: 100, section size: 5
Vanilla benchmark run 1 ... matrix size: 105, section size: 5
Torch benchmark run 1 ... matrix size: 105, section size: 5
Vanilla benchmark run 1 ... matrix size: 110, section size: 5
Torch benchmark run 1 ... matrix size: 110, section size: 5
Done.
